In [ ]:
# Plotting ----

# Get all required packages

#%%
%load_ext autoreload
%autoreload 2

# Packages
import os
import rioxarray as rio
import pandas as pd
import geopandas as gpd

# Local modules
from mod_ra1_plots import *

In [ ]:
# Option

do_figs = False

In [ ]:
# Directories
input_dir = '../data_input/'
output_dir = '../data_output/'

# Choose the list of inversion data version to analyse. Later (shorter) time series are driven by data from more stations.
inversions = {'s76':1976, 's81':1981, 's85':1985, 's99':1999, 's06':2006, 's10':2010}  # Inversionversion and starting year
# inversions = {'s99':1999}  # Inversion starting year. Options: s85, s99, s06, s10

# Number of stations of each inversion version
inv_nstation = {'s76':1976, 's81':1981, 's85':1985, 's99':1999, 's06':2006, 's10':2010}

# List of strating years to define analysis periods
periods = [(1976,2021), (1981,2021), (1985,2021), (1999,2021), (2006,2021), (2010,2021)]
# periods = [(1999,2021)]

In [ ]:
if do_figs:
    
    for inv in inversions:

        inv_start = inversions[inv]

        file_in_amp = os.path.join(output_dir, 'co2fluxAmp_Inv'+str(inv_start)+'.nc')
        co2fluxAmp = rio.open_rasterio(file_in_amp)

        # Plot the first band of co2 flux rates
        # plotOrthographic(xr_val=co2fluxAmp.isel(year=0), ident=inv)

        # Plotting of seasonal amplitude time series
        plotLatMeans(xr_val=co2fluxAmp, lats=([30, 50], [50, 70], [70, 90]), ident='Inv'+str(inv_start))

        for period in periods:

            print('inv_start: ', inv_start)
            print('period: ', period)
        
            if(inv_start > period[0]):
                print('Continuing ...')
                continue

            print('Reading data: ', inv, ', period: ', period)

            ident = str(period[0])+'-'+str(period[1])+'_Inv'+str(inv_start)
            file_in_stats = os.path.join(output_dir, 'co2fluxAmpStats_'+ident+'.nc')
            co2fluxAmpStats = rio.open_rasterio(file_in_stats)

            # Map plot of flux trends in the northern hemisphere
            plotLambert(xarray=co2fluxAmpStats.slope, ident=ident)

            # Plot values with significance filter
            plotOrthoSig(xr_val=co2fluxAmpStats.slope, xr_pval=co2fluxAmpStats.p_value, p_lim=0.1, ident=ident)

            # Plot values without significance filter
            plotOrthoSig(xr_val=co2fluxAmpStats.slope, xr_pval=co2fluxAmpStats.p_value, p_lim=1, ident=ident)

In [ ]:
# Regional time series

# Choose a region (can latter loop through regions)
kopp = rio.open_rasterio('../data_input/VU-VIENA/KG_1986-2010.grd', mask_and_scale=False)
kopp.rio.write_nodata(32, inplace=True)
kopp = kopp.where(kopp != kopp.rio.nodata)
kopp = kopp.rio.reproject("EPSG:4326")

conts = gpd.read_file('../data_input/continents.geojson')


# Create a template dataframe to save results
regind = np.unique(kopp.data[~np.isnan(kopp.data)]).astype(int)
df_regions = pd.DataFrame({'regind': regind, 'regname': kopp_labels,
                          'regcolor': kopp_cols, 'area': np.nan, 'co2amp_var': np.nan})

for inv in inversions:
    
    inv_start = inversions[inv]

    file_in_amp = os.path.join(output_dir, 'co2fluxAmp_Inv'+str(inv_start)+'.nc')
    co2fluxAmp = rio.open_rasterio(file_in_amp)

    regDiffMean = xr_val.where(xr_val.y > lat[0]).where(xr_val.y < lat[1]).mean(['x','y'])


    if inv_start == inversions[0]:
        
        reg_ts = pd.DataFrame(co2fluxAmp)
